In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
import json
import tweetnlp
import openai

2023-11-28 03:34:52,050	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-28 03:34:52,265	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# import functions
import sys
sys.path.append('../Functions/')  # Adjust the relative path as needed

# from json_transform import get_all_keys, flatten_json

In [4]:
''' 
The JSON data is nested, so we need to flatten it to be able to convert it to a DataFrame.
We will use the get_all_keys() and flatten_json() functions from the json_transform.py file.
After getting all the keys, we will convert them to a list and then flatten the JSON data.
Convert the flattened data to a DataFrame and filter out the rows that contain "update" in the text column.
'''
all_tweets = pd.read_csv("../../Data/Processed/Sprint6 - sample_tweets_survey_answers - Ecem.csv")

In [5]:
all_tweets.head(3)

,Timestamp,What is your name?,@TLRailUK why don't the doors open at every station to save us folk having to touch the button?!,"If complaint, whats the topic?",@TLRailUK When on phone to @LNER they said as it was cancelled they've also cancelled ticket and issued refund so now no ticket. That shows on time but all trains going via Croydon before reaching Stevenage so looks like all will be delayed/cancelled,"If complaint, whats the topic?.1","@LiveCommute @bus131 @tlupdates @TLRailUK @brighton_line They are monopolies which go unregulated regarding genuine customer experience, hence three hour train journeys without tables, trays, heating, luggage racks even though they service two of Europes busiest airports. It's a failed industry.","If complaint, whats the topic?.2",@TLRailUK what is going on with the 17:11 from BFR to PLU? We've been on the train for the best part of an hour and are still stuck outside Deptford? Delay repay? Will it ever get to PLU?,"If complaint, whats the topic?.3",...,1012 Thameslink train to Cambridge: Delayed - plat 7 1038 Great Northern train to Moorgate: On time - plat 1 1039 Great Northern train to Hertford North: Cancelled 1022 Thameslink train to Brighton: expected 1042 - plat 2 1043 Great Northern train to Moorgate: Cancelled,"If complaint, whats the topic?.96",TRAVEL: 15 minute delays on @SouthernRailUK @TLRailUK at #Redhill due to a points failure,"If complaint, whats the topic?.97",@TLRailUK What's happening?? Driver hasn't given an announcement in 20 mins - doors are locked and meanwhile trains are going past we could get on,"If complaint, whats the topic?.98",Evening @tlrailuk - you cancel the 1724 ZFD to HHE due to no staff and the 1724 following service is overcrowded with no guard (we had one this morning) to tell that air con is broken - carriage 412114 - rear carriage going south.,"If complaint, whats the topic?.99",@TLRailUK You opened the doors 20 seconds after a train on the parallel platform pulled away. That is absolutely terrible service.,"If complaint, whats the topic?.100"
0,2023/10/31 1:37:36 pm CET,Cem,Complaint,Doors,Complaint,Delays,Complaint,Train General,Complaint,Delay,...,Complaint,Delay,Complaint,Delay,Complaint,"Doors, Announcement, Train General",Non-Complaint,Service,Complaint,Doors
1,2023/10/31 1:37:57 pm CET,ecem,Complaint,Doors,Complaint,Delay,Complaint,Service,Complaint,Delay,...,Complaint,Delay,Complaint,Delay,Complaint,Announcement,Non-Complaint,HVAC,Complaint,Doors
2,2023/11/01 12:28:37 am CET,Shinu J,Complaint,Doors,Uncertain,NaN,Complaint,Absence of amenities,Complaint,Train Delay,...,Complaint,NaN,Non-Complaint,NaN,Complaint,"No announcement, doors",Complaint,"no staff, overcrowding",Complaint,"doors, delays"


In [7]:
# Creating a new DataFrame with the required columns
new_columns = ["text", "Complaint number", "non complaint number", 
               "uncertain number", "Cem", "ecem", "Shinu J", 
               "Akshay", "Erjon"]
survey_edited = pd.DataFrame(columns=new_columns)

# Populate the "text" column with the column names of the complaint statements
text_columns = all_tweets.columns[2::2]  # Selecting every odd-numbered column starting from the 3rd
survey_edited["text"] = text_columns

# Count the number of complaints, non-complaints, and uncertain entries for each text
complaint_counts = []
non_complaint_counts = []
uncertain_counts = []
topics = {name: [] for name in new_columns[4:]}

for text_col in text_columns:
    # Count the occurrences of each type in the corresponding column
    complaint_counts.append((all_tweets[text_col] == "Complaint").sum())
    non_complaint_counts.append((all_tweets[text_col] == "Non-Complaint").sum())
    uncertain_counts.append((all_tweets[text_col] == "Uncertain").sum())

# Iterate over each text column
for text_col in text_columns:
    topic_col = all_tweets.columns[all_tweets.columns.get_loc(text_col) + 1]  # The next column contains the topics
    
    # Reset current_topics for each text column
    current_topics = {reviewer: None for reviewer in topics}
    
    # Iterate over each row in the DataFrame
    for _, row in all_tweets.iterrows():
        reviewer_name = row['What is your name?'].strip().lower()
        for reviewer in topics:
            # Check if the lowercase stripped name matches any reviewer
            if reviewer.lower() == reviewer_name:
                current_topics[reviewer] = row[topic_col]
                break

    # Append the topic for each reviewer
    for reviewer in topics:
        topics[reviewer].append(current_topics[reviewer])

# Now, add the topics to the new DataFrame
for reviewer, topic_list in topics.items():
    survey_edited[reviewer] = topic_list

# Add counts to the new DataFrame
survey_edited["Complaint number"] = complaint_counts
survey_edited["non complaint number"] = non_complaint_counts
survey_edited["uncertain number"] = uncertain_counts

# add the pattern column showing complaint number + non complaint number + uncertain number

survey_edited['pattern'] = survey_edited['Complaint number'].astype(str) + survey_edited['non complaint number'].astype(str) + survey_edited['uncertain number'].astype(str)
survey_edited['pattern'].value_counts()
survey_edited.head(3)

,text,Complaint number,non complaint number,uncertain number,Cem,ecem,Shinu J,Akshay,Erjon,pattern
0,@TLRailUK why don't the doors open at every st...,5,0,0,Doors,Doors,Doors,Door,doors,500
1,@TLRailUK When on phone to @LNER they said as ...,4,0,1,Delays,Delay,NaN,Delay,cancellations,401
2,@LiveCommute @bus131 @tlupdates @TLRailUK @bri...,5,0,0,Train General,Service,Absence of amenities,Customer experience,Experience,500


In [8]:
#Create a copy of the DataFrame named survey_edited
all_tweets_edited = survey_edited.copy()

In [9]:
survey_edited["pattern"].value_counts()

500    30
230    13
401    10
410    10
302     9
050     8
140     7
311     3
041     3
113     2
221     2
023     1
131     1
203     1
320     1
Name: pattern, dtype: int64

In [10]:
# Print the pattern distribution
print('The number of tweets in which we all agree is a complaint: ', len(survey_edited[survey_edited['pattern'].str.contains('500')]))
print('The number of tweets in which we all agree is not a complaint: ', len(survey_edited[survey_edited['pattern'].str.contains('050')]))
print('The number of tweets in which its 4 votes for a complaint: ', len(survey_edited[survey_edited['pattern'].str.contains('410|401')]))
print('The number of tweets in which its 4 votes for a non complaint: ', len(survey_edited[survey_edited['pattern'].str.contains('140|041')]))
print('The number of tweets where there is no agreement: ', len(survey_edited[survey_edited['pattern'].str.contains('3|2')]))

The number of tweets in which we all agree is a complaint:  30
The number of tweets in which we all agree is not a complaint:  8
The number of tweets in which its 4 votes for a complaint:  20
The number of tweets in which its 4 votes for a non complaint:  10
The number of tweets where there is no agreement:  33


In [11]:
#Create a column with the name of sentiment_by_survey that contains True False column where pattern is 500, 410 or 401
survey_edited['sentiment_by_survey'] = np.where(survey_edited['pattern'].str.contains('500|410|401'), True, False)
survey_edited.head(1)

,text,Complaint number,non complaint number,uncertain number,Cem,ecem,Shinu J,Akshay,Erjon,pattern,sentiment_by_survey
0,@TLRailUK why don't the doors open at every st...,5,0,0,Doors,Doors,Doors,Door,doors,500,True


In [12]:
# filter out the rows that contain "update" in the text column of all_tweets_edited
all_tweets_edited = all_tweets_edited[~all_tweets_edited['text'].str.contains("update")]
number_total_tweets = len(all_tweets_edited)

#Printing tweet length after filtering out the rows that contain "update" in the text column
print("Number of total tweets after filtering out the rows that contain 'update' in the text column of modelling dataset: ", number_total_tweets)

Number of total tweets after filtering out the rows that contain 'update' in the text column of modelling dataset:  94


In [13]:
# filter out the rows that contain "update" in the text column of survey_edited
survey_edited = survey_edited[~survey_edited['text'].str.contains("update")]
number_total_tweets = len(survey_edited)

#Printing tweet length after filtering out the rows that contain "update" in the text column
print("Number of total tweets after filtering out the rows that contain 'update' in the text column of survey dataset: ", number_total_tweets)

Number of total tweets after filtering out the rows that contain 'update' in the text column of survey dataset:  94


In [14]:
# filter duplicated rows of all_tweets_edited
all_tweets_edited = all_tweets_edited.drop_duplicates(subset=['text'])
number_total_tweets = len(all_tweets_edited)

#Printing tweet length after filtering out the duplicated rows
print("Number of total tweets after filtering out the duplicated rows of modelling dataset: ", number_total_tweets)

# filter duplicated rows of survey_edited
survey_edited = survey_edited.drop_duplicates(subset=['text'])
number_total_tweets = len(survey_edited)

#Printing tweet length after filtering out the duplicated rows
print("Number of total tweets after filtering out the duplicated rows of survey dataset: ", number_total_tweets)

Number of total tweets after filtering out the duplicated rows of modelling dataset:  94
Number of total tweets after filtering out the duplicated rows of survey dataset:  94


In [15]:
# filter out the rows that contain "delay" in the text column of all_tweets_edited
all_tweets_edited = all_tweets_edited[~all_tweets_edited['text'].str.contains("delay")]
number_total_tweets = len(all_tweets_edited)

#Printing tweet length after filtering out the rows that contain "delay" in the text column
print("Number of total tweets after filtering out the rows that contain 'delay' in the text column of modelling dataset: ", number_total_tweets)

# filter out the rows that contain "delay" in the text column of survey_edited
survey_edited = survey_edited[~survey_edited['text'].str.contains("delay")]
number_total_tweets = len(survey_edited)

#Printing tweet length after filtering out the rows that contain "delay" in the text column
print("Number of total tweets after filtering out the rows that contain 'delay' in the text column of survey dataset: ", number_total_tweets)

Number of total tweets after filtering out the rows that contain 'delay' in the text column of modelling dataset:  55
Number of total tweets after filtering out the rows that contain 'delay' in the text column of survey dataset:  55


In [16]:
survey_edited.head(1)

,text,Complaint number,non complaint number,uncertain number,Cem,ecem,Shinu J,Akshay,Erjon,pattern,sentiment_by_survey
0,@TLRailUK why don't the doors open at every st...,5,0,0,Doors,Doors,Doors,Door,doors,500,True


In [17]:
#Check for sentiment in the all_tweets_edited dataset using the tweetnlp library and save the results in a new column called sentiment
model = tweetnlp.load_model('sentiment')
#Create a new dataframe called transformed_tweets_sentiment as a copy of all_tweets_edited
transformed_tweets_sentiment = all_tweets_edited.copy()
transformed_tweets_sentiment['sentiment'] = transformed_tweets_sentiment['text'].apply(lambda x: model.predict(x)['label'])

# Filter out rows that contaion "negative" in the sentiment column
transformed_tweets_sentiment = transformed_tweets_sentiment[~transformed_tweets_sentiment['sentiment'].str.contains("negative")]
number_total_tweets = len(transformed_tweets_sentiment)

#Printing tweet length after filtering out the rows that contain "negative" in the sentiment column
print("Number of total tweets after filtering out the rows that contain 'negative' in the sentiment column: ", number_total_tweets)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of total tweets after filtering out the rows that contain 'negative' in the sentiment column:  23


In [18]:
#Create a column named sentiment_by_model that contains True if the sentiment is negative and False if the sentiment is positive to all_tweets_edited
transformed_tweets_sentiment['sentiment_by_model'] = np.where(transformed_tweets_sentiment['sentiment'].str.contains('negative'), False, True)
transformed_tweets_sentiment.head(1)    

,text,Complaint number,non complaint number,uncertain number,Cem,ecem,Shinu J,Akshay,Erjon,pattern,sentiment,sentiment_by_model
0,@TLRailUK why don't the doors open at every st...,5,0,0,Doors,Doors,Doors,Door,doors,500,neutral,True


In [19]:
# Copy sentiment_by_model column to survey_edited dataframe regarding the text column
survey_edited['sentiment_by_model'] = transformed_tweets_sentiment['sentiment_by_model'].where(transformed_tweets_sentiment['text'].isin(survey_edited['text']))
survey_edited.head(1)

,text,Complaint number,non complaint number,uncertain number,Cem,ecem,Shinu J,Akshay,Erjon,pattern,sentiment_by_survey,sentiment_by_model
0,@TLRailUK why don't the doors open at every st...,5,0,0,Doors,Doors,Doors,Door,doors,500,True,True


In [20]:
#Fill the NaN values of sentiment_by_model column with False
survey_edited['sentiment_by_model'].fillna(False, inplace=True)

In [21]:
#Print text column of transformed_tweets_sentiment
transformed_tweets_sentiment['text']

0     @TLRailUK why don't the doors open at every st...
6     @soran1 @TLRailUK Drivers are not in charge of...
16    We are delighted to announce our newest partne...
18    @TLRailUK The 0847 from Denmark hill to Blackf...
22    @TLRailUK 0930 at ECR (on late 0807 BTN to LBG...
24    @CinqueBrowne @TLRailUK Traksy, opentraintimes...
25    @charliebrief @TLRailUK Hi. I drive these trai...
32    #Southern Delays of up to ten minutes and some...
36    @TLRailUK Unfortunately not as i am now home. ...
42    @xeepo @TLRailUK @TLRailUK Last month, and man...
47    @TLRailUK @NorthHertsSam @hitchincommuter @Ste...
50    @guy1logic @TLRailUK @mikecurtisg Station staf...
61    @TLRailUK hi there - I've had a proper â€œblok...
65    @TLRailUK hello I live next to Blackfriars. Yo...
66    As his 70th #birthday approaches, #Borehamwood...
73    0821 Thameslink train to Cambridge: Delayed 08...
74    @TLRailUK Streatham. I have now gone to Blackf...
82    @TLRailUK just heard an announcement at BU

In [22]:
survey_edited.head(1)

,text,Complaint number,non complaint number,uncertain number,Cem,ecem,Shinu J,Akshay,Erjon,pattern,sentiment_by_survey,sentiment_by_model
0,@TLRailUK why don't the doors open at every st...,5,0,0,Doors,Doors,Doors,Door,doors,500,True,True


In [23]:
# find the total number of True values in the sentiment_by_survey column
number_of_true = survey_edited['sentiment_by_survey'].value_counts()[1]
print("Number of True values in the sentiment_by_survey column: ", number_of_true)

#Print text column of survey_edit where sentiment_by_survey is True
survey_edited[survey_edited['sentiment_by_survey'] == True]['text']


Number of True values in the sentiment_by_survey column:  27


0      @TLRailUK why don't the doors open at every st...
3      @TLRailUK what is going on with the 17:11 from...
6      @soran1 @TLRailUK Drivers are not in charge of...
18     @TLRailUK The 0847 from Denmark hill to Blackf...
22     @TLRailUK 0930 at ECR (on late 0807 BTN to LBG...
25     @charliebrief @TLRailUK Hi. I drive these trai...
36     @TLRailUK Unfortunately not as i am now home. ...
42     @xeepo @TLRailUK @TLRailUK Last month, and man...
45     for nothing! @TLRailUK No paper in the toilet ...
46     What a total joke on starting my hen do @LNER ...
49     @TLRailUK hi can you end the disruption to my ...
50     @guy1logic @TLRailUK @mikecurtisg Station staf...
52     Delightful @TLRailUK train this morning - fill...
53     Oh great, looks like @TLRailUK on-train WiFi d...
56     @TLRailUK Not according to everyone on the pla...
58     @TLRailUK Not according to everyone on the pla...
63     Also, on a crappy cattle-truck Thameslink trai...
65     @TLRailUK hello I live n

In [24]:
# Print TP, FP, TN, FN values
TP = ((survey_edited['sentiment_by_survey'] == True) & (survey_edited['sentiment_by_model'] == True)).sum()
FP = ((survey_edited['sentiment_by_survey'] == False) & (survey_edited['sentiment_by_model'] == True)).sum()
TN = ((survey_edited['sentiment_by_survey'] == False) & (survey_edited['sentiment_by_model'] == False)).sum()
FN = ((survey_edited['sentiment_by_survey'] == True) & (survey_edited['sentiment_by_model'] == False)).sum()

print("True Positive: ", TP)
print("False Positive: ", FP)
print("True Negative: ", TN)
print("False Negative: ", FN)

True Positive:  15
False Positive:  8
True Negative:  19
False Negative:  13


In [25]:
#Calculate precision, recall, specificity, accuracy and F1 score
precision = TP / (TP + FP)
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
accuracy = (TP + TN) / (TP + FP + TN + FN)
f1_score = 2 * precision * recall / (precision + recall)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Specificity: {specificity}')
print(f'Accuracy: {accuracy}')
print(f'F1 score: {f1_score}')

Precision: 0.6521739130434783
Recall: 0.5357142857142857
Specificity: 0.7037037037037037
Accuracy: 0.6181818181818182
F1 score: 0.5882352941176471


In [26]:
#Calculate Prevellence
prevellence = (TP + FN) / (TP + FP + TN + FN)
print(f'Prevellence: {prevellence}')

Prevellence: 0.509090909090909
